In [1]:
import spiketoolkit as st
import spikesorters as ss
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.utils import Epoch

13:11:34 [I] klustakwik KlustaKwik2 version 0.2.6


In [2]:
workshop_path = "/Users/colehurwitz/"
# workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [4]:
sorting = ss.run_klusta(recording)

Klusta units are not unique! Using unique unit ids


In [5]:
num_spikes = 0
for unit_id in sorting.get_unit_ids():
    num_spikes += len(sorting.get_unit_spike_train(unit_id))
print(num_spikes)

5961


In [6]:
st.postprocessing.export_to_phy(recording, sorting, output_folder='phy_results')

In [7]:
renamed_unit_ids = []
for i in range(len(sorting.get_unit_ids())):
    renamed_unit_ids.append(i*2)
print(renamed_unit_ids)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]


In [8]:
subsorting = se.SubSortingExtractor(sorting, renamed_unit_ids=renamed_unit_ids)

In [9]:
metric_info = st.validation.validation_tools.get_quality_metric_data(recording, subsorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_waveforms=False, verbose=False)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
amplitudes = metric_info[2]
channel_map = metric_info[3]
pc_features = metric_info[4]
pc_feature_ind = metric_info[5]

In [10]:
params = {
          "isi_threshold" : 0.0015,
          "min_isi" : 0.000166,
          "num_channels_to_compare" : 13,
          "max_spikes_for_unit" : 500,
          "max_spikes_for_nn" : 10000,
          "n_neighbors" : 4,
          'n_silhouette' : 10000,
          "quality_metrics_output_file" : "metrics.csv",
          "drift_metrics_interval_s" : 51,
          "drift_metrics_min_spikes_per_interval" : 10
         }

In [11]:
epochs_dict = {
    'red': (300000, 960000),
#     'blue': (0, 300000)
}

In [12]:
epochs = []
for key in list(epochs_dict.keys()):
    epoch_name = key
    epoch_start_frame = epochs_dict[key][0]
    epoch_end_frame = epochs_dict[key][1]
    epochs.append(Epoch(name=key, start_time=recording.frame_to_time(epoch_start_frame), end_time=recording.frame_to_time(epoch_end_frame)))
epochs = sorted(epochs, key=lambda epoch: epoch.start_time)

In [13]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, amplitudes=amplitudes,
                                       channel_map=channel_map, pc_features=pc_features, pc_feature_ind=pc_feature_ind, \
                                       params=params, epochs = epochs, cluster_ids=subsorting.get_unit_ids())

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating amplitude cutoff
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
 ▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 24% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating silhouette score
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating drift metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:382: RuntimeWarning: All-NaN slice encountered


In [14]:
len(sorting.get_unit_spike_train(0, 300000, 960000))

52

In [15]:
metrics_pd

,cluster_id,num_spikes,firing_rate,presence_ratio,isi_viol,amplitude_cutoff,isolation_distance,l_ratio,d_prime,nn_hit_rate,nn_miss_rate,silhouette_score,max_drift,cumulative_drift,epoch_name
0,0,52.0,2.521713,0.38,17.150093,0.0,1.173401e+01,0.091342,2.942100,0.397436,0.004069,NaN,0.0,0.0,red
1,2,106.0,5.140414,0.63,0.000000,0.0,6.571386e+01,0.000534,5.848984,0.874214,0.005051,0.284595,0.0,0.0,red
2,4,34.0,1.648812,0.24,0.000000,0.0,1.921788e+02,0.000006,3.976019,0.490196,0.002315,0.166457,0.0,0.0,red
3,6,315.0,15.275759,0.95,0.000000,0.0,1.528222e+02,0.001892,6.219575,0.929101,0.006754,0.300047,0.0,0.0,red
4,8,219.0,10.620290,0.89,0.000000,0.0,8.567065e+01,0.007788,5.067650,0.852359,0.011351,0.142860,0.0,0.0,red
5,10,74.0,3.588591,0.49,0.000000,0.0,5.613241e+01,0.001136,4.511492,0.603604,0.004760,0.098573,0.0,0.0,red
6,12,18.0,0.872901,0.17,0.000000,0.0,2.681449e+14,NaN,1.442065,0.296296,0.002223,0.021139,0.0,0.0,red
7,14,74.0,3.588591,0.51,0.000000,0.0,5.027972e+01,0.000467,3.751379,0.797297,0.004175,0.112176,0.0,0.0,red
8,16,88.0,4.267514,0.60,0.000000,0.0,1.812448e+02,0.000198,8.446336,0.954545,0.002514,0.202304,0.0,0.0,red
9,18,301.0,14.596837,0.94,0.000000,0.0,6.450972e+01,0.002581,4.347424,0.881506,0.011421,0.176131,0.0,0.0,red


In [16]:
firing_info = st.validation.validation_tools.get_firing_times_ids(sorting, recording.get_sampling_frequency())

spike_times = firing_info[0]
spike_clusters = firing_info[1]
total_units = np.max(spike_clusters) + 1 
channel_map = np.arange(recording.get_num_channels())

print(metrics.calculate_firing_rate_and_spikes(spike_times=spike_times, spike_clusters=spike_clusters, total_units=total_units))
print(metrics.calculate_isi_violations(spike_times=spike_times, spike_clusters=spike_clusters, total_units=total_units,
                                 isi_threshold=params['isi_threshold'], min_isi=params['min_isi']))
print(metrics.calculate_presence_ratio(spike_times=spike_times, spike_clusters=spike_clusters, total_units=total_units))

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
(array([ 2.43424871,  4.40165521,  1.66729364, 14.73887576, 10.77071691,
        3.13451204,  1.16710555,  4.00150473,  4.13488822, 14.70552989,
        0.56687984,  6.33571583, 15.87263544,  2.06744411, 17.20647035,
        3.46797077,  2.73436157,  8.13639296,  9.73699485,  2.76770744,
        5.60210663,  4.60173044,  6.50244519,  1.60060189,  7.4694755 ,
       14.53880053, 12.97154451,  8.33646819,  7.1026709 ]), array([ 73., 132.,  50., 442., 323.,  94.,  35., 120., 124., 441.,  17.,
       190., 476.,  62., 516., 104.,  82., 244., 292.,  83., 168., 138.,
       195.,  48., 224., 436., 389., 250., 213.]))
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[14.76469247  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.04221557  0.          5.01493878  0.56638754
  0.          0.          0.          0.          0.          9.75707418
  0.     

In [17]:
epoch_tuple = (300000, 960000)
firing_rates = st.validation.qualitymetrics.compute_firing_rates(sorting, recording.get_sampling_frequency(), \
                                                                 unit_ids=None, epoch_tuple=None)
print(firing_rates)

num_spikes = st.validation.qualitymetrics.compute_num_spikes(sorting, recording.get_sampling_frequency(), \
                                                             unit_ids=None, epoch_tuple=None)

isi_violations = st.validation.qualitymetrics.compute_isi_violations(sorting,  recording.get_sampling_frequency(), isi_threshold=0.0015, \
                                                                     min_isi=0.000166, unit_ids=None, epoch_tuple=None)
print(isi_violations)

presence_ratios = st.validation.qualitymetrics.compute_presence_ratios(sorting,  recording.get_sampling_frequency(), unit_ids=None, epoch_tuple=None)
print(presence_ratios)


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[ 2.43424871  4.40165521  1.66729364 14.73887576 10.77071691  3.13451204
  1.16710555  4.00150473  4.13488822 14.70552989  0.56687984  6.33571583
 15.87263544  2.06744411 17.20647035  3.46797077  2.73436157  8.13639296
  9.73699485  2.76770744  5.60210663  4.60173044  6.50244519  1.60060189
  7.4694755  14.53880053 12.97154451  8.33646819  7.1026709 ]
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[14.76469247  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.04221557  0.          5.01493878  0.56638754
  0.          0.          0.          0.          0.          9.75707418
  0.          0.          0.          0.17984239  0.        ]
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[0.47 0.71 0.4  0.99 0.96 0.56 0.29 0.73 0.68 0.97 0.17 0.83 0.99 0.51
 0.98 0.67 0.56 0.86 0.93 0.52 0.75 0.68 0.86

In [ ]:
metric_info = st.validation.validation_tools.get_quality_metric_data(recording, subsorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_waveforms=False, verbose=False)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
amplitudes = metric_info[2]
channel_map = metric_info[3]
pc_features = metric_info[4]
pc_feature_ind = metric_info[5]
total_units = np.max(spike_clusters) + 1
print(total_units)
spikes_for_nn = min(len(spike_times), params['max_spikes_for_nn'])
spikes_for_silhouette = min(len(spike_times), params['n_silhouette'])
epoch = Epoch('complete_session', 0, np.inf)
in_epoch = np.logical_and(spike_times > epoch.start_time, spike_times < epoch.end_time)

# print(metrics.calculate_drift_metrics(spike_times, spike_clusters, total_units, pc_features, pc_feature_ind,
#                                       params['drift_metrics_interval_s'], params['drift_metrics_min_spikes_per_interval']))

# print(metrics.calculate_silhouette_score(spike_clusters,
#                                          total_units,
#                                          pc_features,
#                                          pc_feature_ind,
#                                          spikes_for_silhouette))

print(metrics.calculate_pc_metrics(spike_clusters,
                                   total_units,
                                   channel_map,
                                   pc_features,
                                   pc_feature_ind,
                                   params['num_channels_to_compare'],
                                   params['max_spikes_for_unit'],
                                   spikes_for_nn,
                                   params['n_neighbors'],
                                   seed=0)[0])

print(metrics.calculate_pc_metrics(spike_clusters,
                                   total_units,
                                   channel_map,
                                   pc_features,
                                   pc_feature_ind,
                                   params['num_channels_to_compare'],
                                   params['max_spikes_for_unit'],
                                   spikes_for_nn,
                                   params['n_neighbors'],
                                   seed=0)[1])

print(metrics.calculate_pc_metrics(spike_clusters,
                                   total_units,
                                   channel_map,
                                   pc_features,
                                   pc_feature_ind,
                                   params['num_channels_to_compare'],
                                   params['max_spikes_for_unit'],
                                   spikes_for_nn,
                                   params['n_neighbors'],
                                   seed=0)[2])

29
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[9.55575682e+00 5.91247491e+01 5.66048275e+01 1.50119146e+02
 8.95853161e+01 3.84104626e+01 2.71632168e+01 4.83914367e+01
 1.46477883e+02 6.53638227e+01 2.91432144e+14 2.66915684e+01
 1.34286002e+02 1.43089205e+02 8.02410467e+01 1.77268190e+02
 3.83242765e+01 5.27417211e+01 7.96236116e+01 6.44216061e+02
 4.80395610e+02 3.28247199e+02 5.45684586e+01 1.93045312e+01
 1.21339246e+02 1.20556649e+02 1.02812274e+02 4.62539621e+01
 4.19203026e+01]
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[1.50196307e-01 6.26536329e-04 3.63775002e-04 1.85367238e-03
 8.48879103e-03 5.08570840e-03 5.66568978e-03 6.05503935e-04
 1.92861137e-04 2.48492162e-03            nan 1.02927604e-01
 4.49100886e-04 1.70862404e-04 2.80102334e-03 1.21261888e-04
 1.87778838e-02 6.63000189e-03 9.67585356e-04 1.56546200e-03
 3.20154944e-05 1.55757686e-04 4.36696776e-03 4.90635379e-02
 1.01092734e-03 2.36592114e-03 1.69764706e-03 3.33317198e-03
 8.75687220e-03]
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░ 86% 

In [ ]:
epoch_tuple = (300000, 960000)
# max_drifts, cumulative_drifts = st.validation.qualitymetrics.compute_drift_metrics(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, 
#                                                                                    max_num_waveforms=np.inf, max_num_pca_waveforms=np.inf, save_waveforms=False, \
#                                                                                    drift_metrics_interval_s=51, drift_metrics_min_spikes_per_interval=10, unit_ids=None, \
#                                                                                    epoch_tuple=None)
# print(max_drifts, cumulative_drifts)
# silhouette_scores = st.validation.qualitymetrics.compute_silhouette_score(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, 
#                                                                           max_num_waveforms=np.inf, max_num_pca_waveforms=np.inf, save_waveforms=False, \
#                                                                           unit_ids=None, epoch_tuple=None)
# print(silhouette_scores)

isolation_distances = st.validation.qualitymetrics.compute_isolations_distances(recording, sorting, num_channels_to_compare=13, max_spikes_for_unit=500, nPC=3, \
                                                                                ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, max_num_pca_waveforms=np.inf, \
                                                                                save_waveforms=False, unit_ids=None, epoch_tuple=None, seed=0)
print(isolation_distances)

l_ratios = st.validation.qualitymetrics.compute_l_ratios(recording, sorting, num_channels_to_compare=13, max_spikes_for_unit=500, nPC=3, \
                                                         ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, max_num_pca_waveforms=np.inf, \
                                                         save_waveforms=False, unit_ids=None, epoch_tuple=None, seed=0)
print(l_ratios)

d_primes = st.validation.qualitymetrics.compute_d_primes(recording, sorting, num_channels_to_compare=13, max_spikes_for_unit=500, nPC=3, \
                                                         ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, max_num_pca_waveforms=np.inf, \
                                                         save_waveforms=False, unit_ids=None, epoch_tuple=None, seed=0)
print(d_primes)

In [ ]:
# metrics_iso

In [ ]:
metrics.calculate_pc_metrics